In [36]:
%pip install -q transformers datasets evaluate accelerate gluonts ujson

Note: you may need to restart the kernel to use updated packages.


In [1]:
from torch.utils.data import DataLoader
# from loader.custom_dataset import DS, collate_func
from glob import glob
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

from gluonts.time_feature import get_lags_for_frequency, time_features_from_frequency_str
from datasets import load_dataset, Dataset, DatasetDict, DatasetInfo

import os
from plotly.subplots import make_subplots
import plotly.graph_objects as go

/home/ubuntu/Fin/Finance/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# custom gluonts

# Sampling Cell

In [3]:
# OHLC 변환 및 저장
from datetime import time, datetime
parquet_files = glob(os.path.join(os.path.abspath(os.path.pardir), "data", "raw", "*.parquet"))

freq = "1s"
prediction_length = 48

window_size=(3, 0)
sliding_size=(0, 30)
target_size=(1, 0)
preproc = []
def calculate_prob_distribution(data):
    # 1% 상승한 포인트 수
    true_count = np.sum(data)

    # True의 확률
    true_prob = true_count / data.size

    # False의 확률
    false_prob = 1 - true_prob
    
    return true_prob, false_prob # input의 클래스 확률 분포 (True, False)

for idx, parquet_file in enumerate(parquet_files):
    try:
        start_idx = 0
        df = pd.read_parquet(parquet_file)
        start_time = df['STCK_CNTG_HOUR'].iloc[0]
        date_list = df['STCK_CNTG_HOUR'].dt.date.unique()
        
    except IndexError:
        raise StopIteration
    
    except StopIteration:
        break
    
    if start_time.time() < time(9, 0, 0):
        end_time = datetime.combine(start_time.date(), time(9, 0, 0))
        
        x = df[(df['STCK_CNTG_HOUR'] >= start_time) & (df['STCK_CNTG_HOUR'] < end_time)]
        y = df[(df['STCK_CNTG_HOUR'] >= end_time) & (df["STCK_CNTG_HOUR"] <= end_time + target_size)]
        
        # 다음 슬라이딩 시작 시간으로 업데이트
        start_time = end_time
        # 다음 슬라이딩 윈도우의 시작 인덱스 업데이트
        start_idx = df.index[df['STCK_CNTG_HOUR'] >= start_time].min()
        
    elif start_time >= datetime.combine(start_time.date(), time(15, 20, 0)) - target_size: # 
        try:
            end_time = datetime.combine(date_list[np.where(date_list > start_time.date())][0], time(9, 0, 0))
        except IndexError:
            break
        
        x = df[(df['STCK_CNTG_HOUR'] >= start_time) & (df['STCK_CNTG_HOUR'] < end_time)]
        y = df[(df['STCK_CNTG_HOUR'] >= end_time) & (df["STCK_CNTG_HOUR"] <= end_time + target_size)]
        
        # 다음 슬라이딩 시작 시간으로 업데이트
        start_time = end_time
        # 다음 슬라이딩 윈도우의 시작 인덱스 업데이트
        start_idx = df.index[df['STCK_CNTG_HOUR'] >= start_time].min()

    else:
        end_time = start_time + window_size
        x = df[(df['STCK_CNTG_HOUR'] >= start_time) & (df['STCK_CNTG_HOUR'] < end_time)]
        y = df[(df['STCK_CNTG_HOUR'] >= end_time) & (df['STCK_CNTG_HOUR'] <= end_time + target_size)]
        
        # 다음 슬라이딩 시작 시간으로 업데이트
        start_time += sliding_size
        # 다음 슬라이딩 윈도우의 시작 인덱스 업데이트
        start_idx = df.index[df['STCK_CNTG_HOUR'] >= start_time].min()
            
    if x.empty:
        continue
    
    y = calculate_prob_distribution(((y["STCK_PRPR"] - x.iloc[-1]["STCK_PRPR"]) / x.iloc[-1]["STCK_PRPR"] >= 0.01).values)
    preproc.append([x, y])
    break


TypeError: unsupported operand type(s) for +: 'datetime.datetime' and 'tuple'

In [149]:
# OHLC 변환 및 저장
from datetime import time
parquet_files = glob(os.path.join(os.path.abspath(os.path.pardir), "data", "raw", "*.parquet"))

freq = "1s"
prediction_length = 48

window_size=(3, 0)
sliding_size=(0, 30)
target_size=(1, 0)

for idx, parquet_file in enumerate(parquet_files):
    df = pd.read_parquet(parquet_file)
    df = df.assign(ask_volume=df[(df["CCLD_DVSN"] == 1) & (df["STCK_CNTG_HOUR"].dt.time < time(15,20,0)) & (df["STCK_CNTG_HOUR"].dt.time >= time(9,0,0))]["CNTG_VOL"]).fillna(0)
    df = df.assign(bid_volume=df[(df["CCLD_DVSN"] == 5) & (df["STCK_CNTG_HOUR"].dt.time < time(15,20,0)) & (df["STCK_CNTG_HOUR"].dt.time >= time(9,0,0))]["CNTG_VOL"]).fillna(0)
    df = df.assign(after_volume=df[(df["CCLD_DVSN"] == 3) | (df["STCK_CNTG_HOUR"].dt.time >= time(15,20,0)) | (df["STCK_CNTG_HOUR"].dt.time < time(9,0,0))]["CNTG_VOL"]).fillna(0)

    resampler = df.resample(on="STCK_CNTG_HOUR", rule="1s")
    
    re = resampler.agg({
        "STCK_PRPR": "ohlc",
        "CNTG_VOL" : "sum",
        "ask_volume" : "sum",
        "bid_volume" : "sum",
        "after_volume" : "sum"})
    
    re.columns = [col[1] for col in re.columns] # 튜플 colunm 해체
    re[["open", "high", "close", "low"]] = re[["open", "high", "close", "low"]].ffill()
    
    re = re.assign(ticks=resampler.size())
    re = re[(re.index.time >= time(8,30,0))&(re.index.time <= time(16,0,0))]

    re["start"] = re[re.index < (re.index + pd.Timedelta(seconds=180))]

    re.to_parquet(parquet_file.replace("raw", "resample"))


## No Sampling cell, 중복된 timestamp에 대해 ms값 추가

In [2]:
def generate_unique_timestamps(group):
    if len(group) > 1:  # Check if there are duplicates
        # Generate a range of microseconds to add, ensuring uniqueness within the group
        microseconds = pd.to_timedelta(np.arange(len(group))*len(group), unit='ms')
        # print("##############us##############\n", microseconds)
        # print("##############us basic##############\n",pd.to_timedelta(np.arange(len(group)), unit='ms'))
        group['STCK_CNTG_HOUR'] += microseconds
    return group

In [11]:
from functools import partial

parquet_files = glob(os.path.join(os.path.abspath(os.path.pardir), "data", "raw", "*.parquet"))


for idx, parquet_file in enumerate(sorted(parquet_files)):
    df = pd.read_parquet(parquet_file)
    print(df)
    df = df.groupby("STCK_CNTG_HOUR").apply(generate_unique_timestamps).reset_index(drop=True)
    print(df)
    break
    df.to_parquet(parquet_file.replace("raw", "pre"))

       MKSC_SHRN_ISCD      STCK_CNTG_HOUR  STCK_PRPR  CNTG_VOL  CCLD_DVSN
0              005490 2023-09-11 08:30:00   583000.0       1.0        3.0
1              005490 2023-09-11 08:30:00   583000.0       1.0        3.0
2              005490 2023-09-11 08:30:00   583000.0      30.0        3.0
3              005490 2023-09-11 08:30:00   583000.0       3.0        3.0
4              005490 2023-09-11 08:30:00   583000.0       3.0        3.0
...               ...                 ...        ...       ...        ...
714727         005490 2023-10-04 15:56:55   511000.0       5.0        5.0
714728         005490 2023-10-04 15:57:42   511000.0       3.0        5.0
714729         005490 2023-10-04 15:57:55   511000.0      10.0        5.0
714730         005490 2023-10-04 15:58:39   511000.0       1.0        5.0
714731         005490 2023-10-04 15:58:46   511000.0       4.0        5.0

[714732 rows x 5 columns]


KeyboardInterrupt: 

## Try 1

In [126]:
# 'target' 열을 계산하는 함수
from datetime import datetime, time

def compute_target(row):
    if row["start"].time() < time(9,0,0):
        src_start = row["start"]
        src_end = datetime.combine(src_start.date(), time(9,0,0))
        trg_start = src_end
        trg_end = trg_start + pd.Timedelta(seconds=30)
        
    elif row["start"].time() >= time(15, 20, 0):
        src_start = row["start"]
        src_end = datetime.combine(src_start.date() + pd.Timedelta(days=1), time(9,0,0))
        trg_start = src_end
        trg_end = trg_start + pd.Timedelta(seconds=30)  # 그로부터 1분 후
    else:
        src_start = row["start"]
        src_end = src_start + pd.Timedelta(minutes=3)
        trg_start = src_end  # 각 행의 시간으로부터 3분 후
        trg_end = trg_start + pd.Timedelta(seconds=30)

    # trg_start과 trg_end 사이의 데이터 필터링
    src_mask = (df['start'] >= src_start) & (df['start'] <= src_end)
    trg_mask = (df['start'] > trg_start) & (df['start'] < trg_end)
    
    filtered_src = df.loc[src_mask].values if len(df.loc[src_mask]) > 0 else np.array([None])
    filtered_trg = df.loc[trg_mask]["per"].values if len(df.loc[trg_mask]["per"]) > 0 else np.array([0.])

    # 필터링된 데이터를 리스트로 변환하여 반환
    return (filtered_src, filtered_trg)

In [ ]:
parquet_files = glob(os.path.join(os.path.abspath(os.path.pardir), "data", "raw", "*.parquet"))

dataset = DatasetDict({'train': {}, 'validation': {}, "test": {}})
    
##############
freq = "1s"
lags_sequence = get_lags_for_frequency(freq)
time_features = time_features_from_frequency_str(freq)
##############

fig = make_subplots(rows=6, cols=1)

for idx, parquet_file in enumerate(parquet_files):
    df = pd.read_parquet(parquet_file)
    df.rename(columns={"STCK_CNTG_HOUR":"start"}, inplace=True)
    df["per"] = df["STCK_PRPR"] - df["STCK_PRPR"].shift(fill_value=0)
    df["STCK_PRPR"] = df["STCK_PRPR"] * df["CNTG_VOL"]
    df.rename(columns={"STCK_PRPR":"amount"}, inplace=True)
    src, trg = df.apply(compute_target, axis=1)
    print(src)
    print(trg)
    break
    # df["src"] = src
    # df["trg"] = trg

    # 시계열 Features
    timestamp_as_index = pd.DatetimeIndex(df["start"])
    additional_features = [
        (time_feature.__name__, time_feature(timestamp_as_index))
        for time_feature in time_features
    ]
    
    time_features = pd.DataFrame(dict(additional_features))
    train_dataset.set_transform(partial(transform_start_field, freq=freq))
    print(time_features.head())
    print(df.head())



print(dataset)
# ds = DatasetDict({train})

## Try 2

In [ ]:
rom functools import lru_cache

import pandas as pd
import numpy as np



def convert_to_pandas_period(date, freq):
    return pd.Period(date, freq)


def transform_start_field(batch, freq):
    batch["start"] = [convert_to_pandas_period(date, freq) for date in batch["start"]]
    return batch

In [ ]:
# 'target' 열을 계산하는 함수
from datetime import datetime, time

def compute_target(row):   
    src_start = row["start"]
    src_end = src_start + pd.Timedelta(minutes=3)
    trg_start = src_end  # 각 행의 시간으로부터 3분 후
    trg_end = trg_start + pd.Timedelta(seconds=30)

    # trg_start과 trg_end 사이의 데이터 필터링
    src_mask = (df['start'] >= src_start) & (df['start'] <= src_end)
    trg_mask = (df['start'] > trg_start) & (df['start'] < trg_end)
    
    filtered_src = df.loc[src_mask].values if len(df.loc[src_mask]) > 0 else np.array([None])
    filtered_trg = df.loc[trg_mask]["per"].values if len(df.loc[trg_mask]["per"]) > 0 else np.array([0.])

    return filtered_trg

    
def compute_src(row):
    src_start = row["start"]
    src_end = src_start + pd.Timedelta(minutes=3)

    # trg_start과 trg_end 사이의 데이터 필터링
    src_mask = (df['start'] >= src_start) & (df['start'] <= src_end)
    
    filtered_src = df.loc[src_mask].values if len(df.loc[src_mask]) > 0 else np.array([None])
    return filtered_src
    re["start"] = re[re.index < (re.index + pd.Timedelta(seconds=180))]

In [8]:
from sklearn.model_selection import train_test_split
from datasets import Dataset
parquet_files = glob(os.path.join(os.path.abspath(os.path.pardir), "data", "resample", "*.parquet"))

dataset = DatasetDict({'train': {}, 'validation': {}, "test": {}})
    
##############
freq = "1s"
lags_sequence = get_lags_for_frequency(freq)
time_features = time_features_from_frequency_str(freq)
##############

fig = make_subplots(rows=6, cols=1)

for idx, parquet_file in enumerate(parquet_files):
    df = pd.read_parquet(parquet_file)
    df = df.reset_index()
    df = df.rename(columns={"STCK_CNTG_HOUR":"start"})
    # df["target"] = df.loc[df["start"]:(df["start"]+pd.Timedelta(minutes=3))]
    print(df.resample(on="start", rule="1s").size())
    # df["target"] = df.apply(compute_target)
    # df["src"] = df.apply(compute_src)

    # train, valid = train_test_split(df, test_size=0.2, random_state=32)
    # valid, test = train_test_split(valid, test_size=0.2, random_state=32)

    # dataset["train"][idx] = Dataset.from_pandas(train)
    # dataset["validation"][idx] = Dataset.from_pandas(valid)
    # dataset["test"][idx] = Dataset.from_pandas(test)
    # df.rename(columns={"STCK_CNTG_HOUR":"start"}, inplace=True)
    # df["per"] = df["STCK_PRPR"] - df["STCK_PRPR"].shift(fill_value=0)
    # df["STCK_PRPR"] = df["STCK_PRPR"] * df["CNTG_VOL"]
    # df.rename(columns={"STCK_PRPR":"amount"}, inplace=True)
    # src, trg = df.apply(compute_target, axis=1)
    # print(src)
    # print(trg)
    break
    # df["src"] = src
    # df["trg"] = trg

    # 시계열 Features
    timestamp_as_index = pd.DatetimeIndex(df["start"])
    additional_features = [
        (time_feature.__name__, time_feature(timestamp_as_index))
        for time_feature in time_features
    ]
    
    time_features = pd.DataFrame(dict(additional_features))
    train_dataset.set_transform(partial(transform_start_field, freq=freq))
    print(time_features.head())
    print(df.head())



print(dataset)
# ds = DatasetDict({train})

start
2021-05-10 08:30:43    1
2021-05-10 08:30:44    1
2021-05-10 08:30:45    1
2021-05-10 08:30:46    1
2021-05-10 08:30:47    1
                      ..
2021-05-11 15:58:25    1
2021-05-11 15:58:26    1
2021-05-11 15:58:27    1
2021-05-11 15:58:28    1
2021-05-11 15:58:29    1
Freq: S, Length: 113267, dtype: int64
DatasetDict({
    train: {}
    validation: {}
    test: {}
})


In [138]:
dataset["train"][0]

Dataset({
    features: ['start', 'open', 'high', 'low', 'close', 'CNTG_VOL', 'ask_volume', 'bid_volume', 'after_volume', 'ticks', '__index_level_0__'],
    num_rows: 43094
})

## Encoding

In [ ]:

# sys.path.append(os.path.abspath("c:\\Users\\choiseokgyu\\Fin"))
# glob(os.path.join(os.path.abspath(os.path.pardir), "data", "*.parquet"))
# ds = load_dataset(x)
parquet_files = glob(os.path.join(os.path.abspath(os.path.pardir), "data", "raw", "*.parquet"))
##############
freq = "1s"
lags_sequence = get_lags_for_frequency(freq)
time_features = time_features_from_frequency_str(freq)
##############

fig = make_subplots(rows=6, cols=1)
for parquet_file in parquet_files:
    # 현재 Parquet 파일을 DataFrame으로 로드합니다.
    df = pd.read_parquet(parquet_file)
    # df.rename(columns={"STCK_CNTG_HOUR": "start"}, inplace=True)
    # DataFrame을 Dataset으로 변환합니다.
    # dataset = Dataset.from_parquet(path_or_paths=parquet_file)

    prediction_length = 48
    df.rename(columns={"STCK_CNTG_HOUR": "start"}, inplace=True)
    df = df.assign(amount=df["STCK_PRPR"] * df["CNTG_VOL"])
    timestamp_as_index = pd.DatetimeIndex(df["start"])
    additional_features = [
        (time_feature.__name__, time_feature(timestamp_as_index))
        for time_feature in time_features
    ]
    
    # print(np.column_stack(list(dict(additional_features).values())))
    # print(dict(additional_features).keys())
    for idx, (key, val) in enumerate(dict(additional_features).items()):
        # fig.add_trace(go.Scatter(x=np.arange(len(val)), y=val, mode="lines", name=f"{key}"),row=idx+1, col=1)
        df[key] = val
    
    # fig.update_layout(height=1200)
    # fig.show()
    df = df[df.columns.drop(["STCK_PRPR", "MKSC_SHRN_ISCD"])]
    # print(df[df.columns.drop(["STCK_PRPR", "MKSC_SHRN_ISCD"])])
    # print(df)
    # df.to_parquet(parquet_file.replace("raw", "pre"))
    # dataset = 
    # print(df)

    # 변환된 Dataset을 목록에 추가합니다.
    # datasets_list.append(dataset)


In [ ]:
from transformers import InformerConfig, InformerForPrediction

config = InformerConfig(
    # in the multivariate setting, input_size is the number of variates in the time series per time step
    input_size=10997,
    # prediction length:
    prediction_length=prediction_length,
    # context length:
    context_length=prediction_length * 2,
    # lags value copied from 1 week before:
    lags_sequence=[1, 24 * 7],
    # we'll add 5 time features ("hour_of_day", ..., and "age"):
    num_time_features=len(time_features) + 1,
    
    # informer params:
    dropout=0.1,
    encoder_layers=6,
    decoder_layers=4,
    # project input from num_of_variates*len(lags_sequence)+num_time_features to:
    d_model=64,
)

model = InformerForPrediction(config)

In [22]:
from functools import lru_cache, partial

import pandas as pd
import numpy as np

@lru_cache(10_000)
def convert_to_pandas_period(date, freq):
    print(date)
    return pd.Period(date, freq)


def transform_start_field(batch, freq):
    batch["STCK_CNTG_HOUR"] = [convert_to_pandas_period(date, freq) for date in batch["STCK_CNTG_HOUR"]]
    print(batch)
    return batch

parquet_files = glob(os.path.join(os.path.abspath(os.path.pardir), "data", "pre", "*.parquet"))

datasets_list = []
fig = make_subplots(rows=6, cols=1)
for parquet_file in parquet_files:
    # 현재 Parquet 파일을 DataFrame으로 로드합니다.
    df = pd.read_parquet(parquet_file)
    
    # DataFrame을 Dataset으로 변환합니다.
    dataset = Dataset.from_parquet(path_or_paths=parquet_file)
    datasets_list.append(dataset)
datasets_list[0].set_transform(partial(transform_start_field, freq=freq))
datasets_list[0]

Dataset({
    features: ['MKSC_SHRN_ISCD', 'STCK_CNTG_HOUR', 'STCK_PRPR', 'CNTG_VOL', 'CCLD_DVSN', 'second_of_minute', 'minute_of_hour', 'hour_of_day', 'day_of_week', 'day_of_month', 'day_of_year'],
    num_rows: 14288
})

In [35]:
from transformers import InformerModel
import torch
from typing import List, Optional, Tuple, Union
from transformers.modeling_outputs import BaseModelOutput

class CustomInformer(InformerModel):
    def __init__(self, config):
        super().__init__(config)
        self.encoder = InformerEncoder(config)
    
    def forward(
        self,
        past_values: torch.Tensor,
        past_time_features: torch.Tensor,
        past_observed_mask: torch.Tensor,
        static_categorical_features: Optional[torch.Tensor] = None,
        static_real_features: Optional[torch.Tensor] = None,
        future_values: Optional[torch.Tensor] = None,
        future_time_features: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        encoder_outputs: Optional[List[torch.FloatTensor]] = None,
        output_hidden_states: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        use_cache: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ):
        r"""
        Returns:

        Examples:

        ```python
        >>> from huggingface_hub import hf_hub_download
        >>> import torch
        >>> from transformers import InformerModel

        >>> file = hf_hub_download(
        ...     repo_id="hf-internal-testing/tourism-monthly-batch", filename="train-batch.pt", repo_type="dataset"
        ... )
        >>> batch = torch.load(file)

        >>> model = InformerModel.from_pretrained("huggingface/informer-tourism-monthly")

        >>> # during training, one provides both past and future values
        >>> # as well as possible additional features
        >>> outputs = model(
        ...     past_values=batch["past_values"],
        ...     past_time_features=batch["past_time_features"],
        ...     past_observed_mask=batch["past_observed_mask"],
        ...     static_categorical_features=batch["static_categorical_features"],
        ...     static_real_features=batch["static_real_features"],
        ...     future_values=batch["future_values"],
        ...     future_time_features=batch["future_time_features"],
        ... )

        >>> last_hidden_state = outputs.last_hidden_state
        ```"""
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        use_cache = use_cache if use_cache is not None else self.config.use_cache
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        transformer_inputs, loc, scale, static_feat = self.create_network_inputs(
            past_values=past_values,
            past_time_features=past_time_features,
            past_observed_mask=past_observed_mask,
            static_categorical_features=static_categorical_features,
            static_real_features=static_real_features,
            future_values=future_values,
            future_time_features=future_time_features,
        )

        if encoder_outputs is None:
            enc_input = transformer_inputs[:, : self.config.context_length, ...]
            encoder_outputs = self.encoder(
                inputs_embeds=enc_input,
                head_mask=head_mask,
                output_attentions=output_attentions,
                output_hidden_states=output_hidden_states,
                return_dict=return_dict,
            )
            
        # If the user passed a tuple for encoder_outputs, we wrap it in a BaseModelOutput when return_dict=True
        elif return_dict and not isinstance(encoder_outputs, BaseModelOutput):
            encoder_outputs = BaseModelOutput(
                last_hidden_state=encoder_outputs[0],
                hidden_states=encoder_outputs[1] if len(encoder_outputs) > 1 else None,
                attentions=encoder_outputs[2] if len(encoder_outputs) > 2 else None,
            )

        return encoder_outputs

In [43]:
model = CustomInformer(config)
model(**{
    "past_values": torch.rand(1, 10, 9),
    "past_time_features": torch.rand(1, 10, 6),
    "past_observed_mask": torch.rand(1, 9),
    "static_categorical_features": torch.rand(1, 10, 1),
    "static_real_features": torch.rand(1, 10, 1),
    "future_values": torch.rand(1, 10, 9),
    "future_time_features": torch.rand(1, 10, 6),
})

RuntimeError: Tensors must have same number of dimensions: got 3 and 2

In [ ]:
import torch
from torch import nn
import math


def probsparse_attention(query_states, key_states, value_states, sampling_factor=5):
    """
    Compute the probsparse self-attention.
    Input shape: Batch x Time x Channel

    Note the additional `sampling_factor` input.
    """
    # get input sizes with logs
    L_K = key_states.size(1)
    L_Q = query_states.size(1)
    log_L_K = np.ceil(np.log1p(L_K)).astype("int").item()
    log_L_Q = np.ceil(np.log1p(L_Q)).astype("int").item()

    # calculate a subset of samples to slice from K and create Q_K_sample
    U_part = min(sampling_factor * L_Q * log_L_K, L_K)

    # create Q_K_sample (the q_i * k_j^T term in the sparsity measurement)
    index_sample = torch.randint(0, L_K, (U_part,))
    K_sample = key_states[:, index_sample, :]
    Q_K_sample = torch.bmm(query_states, K_sample.transpose(1, 2))

    # calculate the query sparsity measurement with Q_K_sample
    M = Q_K_sample.max(dim=-1)[0] - torch.div(Q_K_sample.sum(dim=-1), L_K)

    # calculate u to find the Top-u queries under the sparsity measurement
    u = min(sampling_factor * log_L_Q, L_Q)
    M_top = M.topk(u, sorted=False)[1]

    # calculate Q_reduce as query_states[:, M_top]
    dim_for_slice = torch.arange(query_states.size(0)).unsqueeze(-1)
    Q_reduce = query_states[dim_for_slice, M_top]  # size: c*log_L_Q x channel

    # and now, same as the canonical
    d_k = query_states.size(-1)
    attn_scores = torch.bmm(Q_reduce, key_states.transpose(-2, -1))  # Q_reduce x K^T
    attn_scores = attn_scores / math.sqrt(d_k)
    attn_probs = nn.functional.softmax(attn_scores, dim=-1)
    attn_output = torch.bmm(attn_probs, value_states)

    return attn_output, attn_scores


In [ ]:
# ConvLayer is a class with forward pass applying ELU and MaxPool1d
def informer_encoder_forward(x_input, num_encoder_layers=3, distil=True):
    # Initialize the convolution layers
    if distil:
        conv_layers = nn.ModuleList([ConvLayer() for _ in range(num_encoder_layers - 1)])
        conv_layers.append(None)
    else:
        conv_layers = [None] * num_encoder_layers
    
    # Apply conv_layer between each encoder_layer
    for encoder_layer, conv_layer in zip(encoder_layers, conv_layers):
        output = encoder_layer(x_input)
        if conv_layer is not None:
            output = conv_layer(loutput)
    
    return output